##### Cálculo de Métricas
######  Garante que os CSVs existem (senão roda a simulação automaticamente).
######  Carrega os DataFrames.
######  Calcula as métricas principais.
######  Retorna e imprime o resumo.

In [29]:
import os
import pandas as pd
import numpy as np

# Importa a Simulação
import import_ipynb
from Simulate_Portfolio import simulate_portfolio

In [30]:
def calculate_metrics(portfolio_df, cashflow_df):
    results = {}

    # 1. Saldo Total
    saldo_total = portfolio_df["principal"].sum()

    # 2. Taxa de Inadimplêcia
    taxa_inadimplencia = portfolio_df["default_month"].notna().mean()

    # 3. Fluxo de Caixa Agregado sem inadimplencia + recovey
    fluxo_caixa_aggreg = (
        cashflow_df
        .groupby("month")["scheduled_payment"]
        .sum()
        .reset_index(name="fluxo_aggregado")
    )

    # 4. Fluxo de caixa real (com inadimplência + recovery)  
    cashflow_df["fluxo_real"] = cashflow_df["scheduled_payment"] + cashflow_df["recovery_cashflow"]  
    fluxo_caixa_real = (  
        cashflow_df  
        .groupby("month")["fluxo_real"]  
        .sum()  
        .reset_index()  
    )

    # 5. Retorno esperado via IRR
    fluxos = [-saldo_total] + fluxo_caixa_real["fluxo_real"].tolist()
    try:
        retorno_esperado_IRR = np.irr(fluxos)
    except Exception:
        retorno_esperado_IRR = np.nan

    results.update({
        "saldo_total": saldo_total,
        "taxa_inadimplencia": taxa_inadimplencia,
        "fluxo_caixa_aggreg": fluxo_caixa_aggreg,
        "fluxo_caixa_real": fluxo_caixa_real,
        "retorno_esperado_IRR": retorno_esperado_IRR
    })

    return results

In [31]:
# Garante que a pasta exista  
os.makedirs("data", exist_ok=True)  
  
# Caminhos  
portfolio_path = "data/portifolio.csv"  
cashflows_path = "data/cashflows.csv"  
  
# Se não houver CSV, gera a simulação  
if not (os.path.exists(portfolio_path) and os.path.exists(cashflows_path)):  
    print("⚠️  CSVs não encontrados. Gerando simulação...")  
    df_portfolio, df_cashflows = simulate_portfolio(n_contracts=10_000, seed=42)  
    df_portfolio.to_csv(portfolio_path, index=False)  
    df_cashflows.to_csv(cashflows_path, index=False)  
else:  
    print("✅ CSVs encontrados. Carregando...")  
    df_portfolio = pd.read_csv(portfolio_path)  
    df_cashflows = pd.read_csv(cashflows_path)

✅ CSVs encontrados. Carregando...


In [32]:
# Calcula métricas  
metrics = calculate_metrics(df_portfolio, df_cashflows)  
  
# Prints  
print("Saldo Total:", metrics["saldo_total"])  
print("Taxa Inadimplência:", metrics["taxa_inadimplencia"])  
print("Retorno Esperado (IRR mensal):", metrics["retorno_esperado_IRR"])  
print("\nFluxo Caixa Agregado:\n", metrics["fluxo_caixa_aggreg"].head())  
print("\nFluxo Caixa Real:\n", metrics["fluxo_caixa_real"].head())

Saldo Total: 380375223.97338593
Taxa Inadimplência: 0.1787
Retorno Esperado (IRR mensal): nan

Fluxo Caixa Agregado:
    month  fluxo_aggregado
0      1     1.280811e+07
1      2     1.275159e+07
2      3     1.270145e+07
3      4     1.264862e+07
4      5     1.257817e+07

Fluxo Caixa Real:
    month    fluxo_real
0      1  1.372814e+07
1      2  1.363886e+07
2      3  1.351314e+07
3      4  1.331599e+07
4      5  1.361498e+07
